In [180]:
import pandas as pd
from py4j.java_gateway import JavaGateway
import psycopg2
import numpy as np

In [181]:
home = "/Users/liviaalmada/NYDataset/Link Level Traffic Estimates/"
home = "/media/livia/DATA/Dropbox/TRABALHO/PESQUISA/Doutorado/Dados/NY-Data/"

In [183]:
graphnodes = pd.read_csv(home+"nodes.csv")

In [184]:
graphlinks = pd.read_csv(home+"/links.csv", delimiter=',')

In [185]:
graphnodes.head()

,node_id,is_complete,num_in_links,num_out_links,osm_traffic_controller,xcoord,ycoord,osm_changeset,birth_timestamp,death_timestamp,grid_region_id
0,42467266,t,3,3,null,-73.917419,40.660051,3218174,1405267167641,40000000000000,225
1,42467262,t,4,4,null,-73.918294,40.659484,3218174,1405267167641,40000000000000,225
2,42467259,t,3,3,null,-73.919089,40.658966,3218174,1405267167641,40000000000000,205
3,42467258,t,3,3,null,-73.919968,40.658398,3218174,1405267167641,40000000000000,205
4,42467250,t,2,2,null,-74.005260,40.600091,7948780,1405267167641,40000000000000,143


In [186]:
graphlinks.head()

,link_id,begin_node_id,end_node_id,begin_angle,end_angle,street_length,osm_name,osm_class,osm_way_id,startX,startY,endX,endY,osm_changeset,birth_timestamp,death_timestamp
0,1,103235840,103225947,-161.51,16.99,84.295,PalisadeAvenue,tertiary,223694294,-74.021807,40.884127,-74.021994,40.883384,16353009,1405267167641,40000000000000
1,2,42516422,42516427,175.46,-4.68,260.392,SchenectadyAvenue,residential,5679907,-73.932264,40.641950,-73.932013,40.639616,19443546,1405267167641,40000000000000
2,3,42516422,42516418,-4.60,175.40,256.804,SchenectadyAvenue,residential,5679907,-73.932264,40.641950,-73.932509,40.644252,19443546,1405267167641,40000000000000
3,4,103235530,103235525,-103.43,82.40,76.107,HarristownRoad,tertiary,11580026,-74.142573,40.948839,-74.143464,40.948718,14802365,1405267167641,40000000000000
4,5,42762376,42756156,-23.92,156.25,197.829,KingslandAvenue,residential,5698286,-73.844794,40.877882,-73.845749,40.879508,8219326,1405267167641,40000000000000


### Create Graph

In [286]:
gateway = JavaGateway() 

In [287]:
gateway.newGraph("ny", home+"graph")

In [288]:
graph = gateway.getGraph("ny");

In [289]:
graph.setNumberOfIntervals(24)

In [290]:
nodesMap = { }


In [291]:
idn = 0
for n in graphnodes.values:
    #gateway.addNode(n[0], n[5], n[6])
    nodesMap[n[0]] = idn
    idn = idn + 1

In [292]:
graph.getNumberOfNodes()

0

In [293]:
print(nodesMap.get(0))

None


In [298]:
nodesMap = dict()
idn = 0
for (idx, row) in graphlinks.iterrows():
    
    if(nodesMap.get(row['begin_node_id']) is None):
        gateway.addNode( row['begin_node_id'] , row['startX'] , row['startY'])
        nodesMap[row['begin_node_id']] = idn
        idn = idn + 1
    if(nodesMap.get(row['end_node_id']) is None):
        gateway.addNode( row['end_node_id'] , row['endX'] , row['endY'])
        nodesMap[row['end_node_id']] = idn
        idn = idn + 1
    gateway.addEdge(nodesMap.get(row['begin_node_id']), nodesMap.get(row['end_node_id']), row['street_length'])
       

In [301]:
graph.getNumberOfEdges()

1304275

In [302]:
graph.getNumberOfNodes()

482850

In [305]:
graphcosts = pd.read_csv(home+"/links.csv", delimiter=',')

In [307]:
graphcosts.head()

,link_id,begin_node_id,end_node_id,begin_angle,end_angle,street_length,osm_name,osm_class,osm_way_id,startX,startY,endX,endY,osm_changeset,birth_timestamp,death_timestamp
0,1,103235840,103225947,-161.51,16.99,84.295,PalisadeAvenue,tertiary,223694294,-74.021807,40.884127,-74.021994,40.883384,16353009,1405267167641,40000000000000
1,2,42516422,42516427,175.46,-4.68,260.392,SchenectadyAvenue,residential,5679907,-73.932264,40.641950,-73.932013,40.639616,19443546,1405267167641,40000000000000
2,3,42516422,42516418,-4.60,175.40,256.804,SchenectadyAvenue,residential,5679907,-73.932264,40.641950,-73.932509,40.644252,19443546,1405267167641,40000000000000
3,4,103235530,103235525,-103.43,82.40,76.107,HarristownRoad,tertiary,11580026,-74.142573,40.948839,-74.143464,40.948718,14802365,1405267167641,40000000000000
4,5,42762376,42756156,-23.92,156.25,197.829,KingslandAvenue,residential,5698286,-73.844794,40.877882,-73.845749,40.879508,8219326,1405267167641,40000000000000


In [308]:
grouped = graphcosts.groupby('begin_node_id')

In [311]:
grouped.ngroups

96435

## Generate graph costs

In [312]:
conn = psycopg2.connect(host="localhost", database="newyork", user="postgres")

In [313]:
curs = conn.cursor()

In [315]:
curs.execute("select begin_node_id, end_node_id, meantt, stdtt, hourday from gaussian_tt where weekday = 0 and begin_node_id<>0")

In [317]:
costs = curs.fetchall()

In [318]:
costs
curs.close()

In [320]:
costs

[]

In [322]:
for cost in costs:
    node_from = nodesMap.get(cost[0])
    node_to = nodesMap.get(cost[1])
    meantt = cost[2]
    stdtt = cost[3]
    hourday = cost[4]
    gateway.addGaussianCost(node_from,node_to,meantt, stdtt, int(hourday))

In [323]:
x = graph.getProbabilisticCosts(1)


In [324]:
gateway.runLinkScanWithKolnogorovSmirnovDistance("ny", home+"link2", 0.5, 5, 20,2)